# Descriptors

Descriptors specify what kinds of values are valid for a parameter.  
There are currently, three variants: `RangeParameter`, `DependentParameter` and `CategoryParameter`.  

In [1]:
import pandas as pd

from besos.parameters import RangeParameter, DependentParameter, CategoryParameter, Parameter
from besos.problem import Problem
from besos import sampling
from besos.evaluator import EvaluatorGeneric 

### RangeParameters 
$min \leq x \leq max$

In [2]:
zero_to_one_exclusive = RangeParameter(min_val = 0.01, max_val=0.99)

### DependentParameter 

The value of dependent parameter depends on a range parameter.

For mode = 'sum', x = factor - target_val

For mode = 'multiple', x = factor * target_val

For mode = 'power', x = target_val ^ factor

index represents the index of the target range parameter in the parameter list

Check out [this](Evaluators/DependParamAndNonObj.ipynb) notebook for more examples.

In [3]:
dp = DependentParameter(mode = 'sum', factor = 1, index = 0)

### CategoryParameters 
A list of options.

In [4]:
text_example = CategoryParameter(options=['a', 'b', 'c', 'other'])
single_digit_integers = CategoryParameter(options=range(10))

### Sampling
These descriptors can be used to make `Parameters`.  
Then we can generate samples.

In [5]:
parameters = [
    Parameter(value_descriptor=zero_to_one_exclusive, name='0-1'),
    Parameter(value_descriptor=dp, name='dp'),
    Parameter(value_descriptor=single_digit_integers, name='single digit'),
    Parameter(value_descriptor=text_example, name='text')
]
problem = Problem(parameters, outputs=['output'])

samples = sampling.dist_sampler(sampling.lhs, problem, num_samples=10)
samples

,0-1,dp,single digit,text
0,0.322092,0.677908,5,other
1,0.763874,0.236126,7,other
2,0.968516,0.031484,9,b
3,0.514495,0.485505,8,a
4,0.193031,0.806969,3,c
5,0.245641,0.754359,1,c
6,0.799233,0.200767,2,a
7,0.432996,0.567004,4,other
8,0.611442,0.388558,6,b
9,0.042449,0.957551,0,a


### Evaluation
Since we did not specify selectors for the parameters, we cannot evaluate them using an EnergyPlus simulation.  
Instead, we will use a custom evaluation function.

In [6]:
def evaluation_function(values):
    v, x, y, z = values
    if z == 'other':
        return (v,), ()
    else:
        return (x * y,), ()

evaluator = EvaluatorGeneric(evaluation_function, problem)
# The evaluator will use this objective by default
outputs = evaluator.df_apply(samples ,keep_input=True)
# outputs is a pandas dataframe with one column since only one objective was requested

In [7]:
outputs

,0-1,dp,single digit,text,output
0,0.322092,0.677908,5,other,0.322092
1,0.763874,0.236126,7,other,0.763874
2,0.968516,0.031484,9,b,0.283360
3,0.514495,0.485505,8,a,3.884041
4,0.193031,0.806969,3,c,2.420908
5,0.245641,0.754359,1,c,0.754359
6,0.799233,0.200767,2,a,0.401535
7,0.432996,0.567004,4,other,0.432996
8,0.611442,0.388558,6,b,2.331347
9,0.042449,0.957551,0,a,0.000000
